## Assignment 1 - Feature Engineering

### Instructions


Credit reports are part of the external data recovered by FairMoney. The point here is to evaluate your ability to process semi-structured data to extract features that you think are relevant for credit scoring.

- From the file *credit_report_sample.json*, the goal is to extract/create all the features that seem relevant to you.

- For each constructed variable, tell us how it could be relevant to improve the risk scoring model.

- The output should be a Python function/class which takes one or more credit reports as input and returns the features as they can be used by a model.



### Thought Process and Intuition

- So we have a nested JSON like file, which is multiple nested with both dictionary, list available inside it. The big question over here is the nested objects could be limitless, they could be n number of such objects. So we can not manually do them

    - Hence a simple for loop or json_load or json_normalize might not able to fetch the results. However we will try them and see the results
    
    - Second, we could think of a recursive function which run until the object inside the nested remains a dictionary or list type 
        - We can start with base prefix as blank for the starting time and recursivel add that base prefix if the object is list or dictionary, if not then the base prefix will be updated with column value.
        

### 1. Imports 

We are going to import the basic library pandas and json, as we are going to do much of our code on that.

In [4]:
import pandas as pd
import json

#pd.set_option('display.max_rows', None)

### 2. Let's try the first approach of json load and json normalize

In [5]:
#load JSON object 
with open('../dat/credit_report_sample.json') as f:
    json_data = json.load(f)

# There is an pandas inbuilt function to normalize the JSON the data or flatten the JSON data
df_json = pd.json_normalize(json_data)
df_json.head(3)

,application_id,data.consumerfullcredit.subjectlist.reference,data.consumerfullcredit.subjectlist.consumerid,data.consumerfullcredit.subjectlist.searchoutput,data.consumerfullcredit.accountrating.noofotheraccountsbad,data.consumerfullcredit.accountrating.noofotheraccountsgood,data.consumerfullcredit.accountrating.noofretailaccountsbad,data.consumerfullcredit.accountrating.noofretailaccountsgood,data.consumerfullcredit.accountrating.nooftelecomaccountsbad,data.consumerfullcredit.accountrating.noofautoloanaccountsbad,...,data.consumerfullcredit.accountmonthlypaymenthistoryheader.mh22,data.consumerfullcredit.accountmonthlypaymenthistoryheader.mh23,data.consumerfullcredit.accountmonthlypaymenthistoryheader.mh24,data.consumerfullcredit.accountmonthlypaymenthistoryheader.company,data.consumerfullcredit.accountmonthlypaymenthistoryheader.tablename,data.consumerfullcredit.accountmonthlypaymenthistoryheader.displaytext,data.consumerfullcredit.identificationhistory,data.consumerfullcredit.personaldetailssummary.worktelephoneno,data.consumerfullcredit.personaldetailssummary.passportno,data.consumerfullcredit.personaldetailssummary.residentialaddress4
0,9711360,128566,128566,XXX,0,3,0,2,0,0,...,2018\nNOV,2018\nOCT,2018\nSEP,Company,Consumer24MonthlyPaymentHeader,Consumer 24 Monthly Payment Header,NaN,NaN,NaN,NaN
1,9714953,58793,58793,XXX,0,3,0,12,0,0,...,2018\nDEC,2018\nNOV,2018\nOCT,Company,Consumer24MonthlyPaymentHeader,Consumer 24 Monthly Payment Header,"[{'updatedate': '21/11/2014', 'updateondate': ...",XXX,NaN,NaN
2,9714978,17688366,17688366,XXX,0,1,1,1,0,0,...,2018\nDEC,2018\nNOV,2018\nOCT,Company,Consumer24MonthlyPaymentHeader,Consumer 24 Monthly Payment Header,NaN,NaN,XXX,Nigeria


> The issue with above approach is that it does not do well where it encounters mix type of nested objects, example a list inside a dictionary or a dictionary inside a list. As see from the below -> consumerfullcredit.identificationhistory ends up as the list

In [9]:
df_json[['data.consumerfullcredit.identificationhistory']]

,data.consumerfullcredit.identificationhistory
0,NaN
1,"[{'updatedate': '21/11/2014', 'updateondate': ..."
2,NaN


### 3. Second appraoch of recursive function

- To handle the above mentioned issue, we should have a function which runs until it finds a integer, string object. That is to say, if the object is dictionary or list then, it will run again and add the base node or prefix to the recursive keys inside the nested JSON object

In [10]:
def convert_json(data, prefix=''):
    """
    Arguments: 
        data : This is the dictionary, or list from the column of dataframe
        prefix = Default -> ''. To se the base prefix or the root node of the dictionary and recursively update it
    Returns:
        rows : A list with key and values for each of the nested key in dictionary or list
    """
    if isinstance(data, dict):
        rows = []
        for key, value in data.items():
            rows += convert_json(value, prefix + key + '.')
        return rows
    elif isinstance(data, list):
        rows = []
        for i, item in enumerate(data):
            rows += convert_json(item, prefix + str(i) + '.')
        return rows
    else:
        return [(prefix[:-1], data)]


### 4. Use of above function in dataframe object with nested JSON

In [12]:
    
# 1. Loading the data to a dataframe
df = pd.read_json('../dat/credit_report_sample.json')

# 2. Empty list for storing the return of the function
rows=[]

# 3. We will first check if the column requires this transformation or not
for col in df.columns:
    if df[col].dtype == 'object':
        for item in df[col]:
            rows.append(dict(convert_json(item)))

    else:
        df_non_flatten = df[col]

# 4. Storing appended rows output as a dataframe
df_flatten = pd.DataFrame(rows)

# 5. Concatenating the two ouput from above loop
df_transformed = pd.concat([df_non_flatten, df_flatten], axis=1)

# 6. Write the output to a csv file
df_transformed.to_csv(r'../dat/json_flatten_transformed.csv')


In [13]:
df_transformed.head(3)

,application_id,consumerfullcredit.subjectlist.reference,consumerfullcredit.subjectlist.consumerid,consumerfullcredit.subjectlist.searchoutput,consumerfullcredit.accountrating.noofotheraccountsbad,consumerfullcredit.accountrating.noofotheraccountsgood,consumerfullcredit.accountrating.noofretailaccountsbad,consumerfullcredit.accountrating.noofretailaccountsgood,consumerfullcredit.accountrating.nooftelecomaccountsbad,consumerfullcredit.accountrating.noofautoloanaccountsbad,...,consumerfullcredit.accountmonthlypaymenthistory.16.dateaccountopened,consumerfullcredit.accountmonthlypaymenthistory.16.openingbalanceamt,consumerfullcredit.accountmonthlypaymenthistory.16.performancestatus,consumerfullcredit.accountmonthlypaymenthistory.16.subscribertypeind,consumerfullcredit.accountmonthlypaymenthistory.16.indicatordescription,consumerfullcredit.accountmonthlypaymenthistory.16.monthlyinstalmentamt,consumerfullcredit.accountmonthlypaymenthistory.16.repaymentfrequencycode,consumerfullcredit.personaldetailssummary.passportno,consumerfullcredit.personaldetailssummary.residentialaddress4,consumerfullcredit.accountmonthlypaymenthistory.0.currentbalancedebitind
0,9711360,128566,128566,XXX,0,3,0,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9714953,58793,58793,XXX,0,3,0,12,0,0,...,31/07/2020,"16,500.00",Performing,F,Personal fixed term loan,"7,232.50",Not Available,NaN,NaN,NaN
2,9714978,17688366,17688366,XXX,0,1,1,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX,Nigeria,NaN


### 6. Which features are important ?

- As we have only sample data have just 3 rows, we can `not` notice the feature by the way of completeness (Is is reguraly updated, No. of missing values ?, Relation with target variable ?, Quantile Plots ?, Descriptive Statistics ? ) 

- However we can just look at the more in `business sense` from their description, and what it means to include in risk scoring model

In [15]:

# 1. Let's store all the columns from our previous flattened JSON to df_features dataframe
df_features = df_transformed.columns.to_frame(index=None, name='features')

# 2. Split the data based on the spearator and expand it to dummy columns
column_names =[]
for i in df_features.features.str.split('.', expand=True).columns:
    column_names.append('node_'+ str(i))

# 3. Store data with column names in df_festure dataframe
df_features[column_names] = df_features.features.str.split('.', expand=True)

# 4. Groupby on the base node which is node_0 and node_1 and check the node_1 value for analysis
df_features.groupby(['node_0', 'node_1']).all().reset_index()

,node_0,node_1,features,node_2,node_3
0,consumerfullcredit,accountmonthlypaymenthistory,True,True,True
1,consumerfullcredit,accountmonthlypaymenthistoryheader,True,True,True
2,consumerfullcredit,accountrating,True,True,True
3,consumerfullcredit,creditaccountsummary,True,True,True
4,consumerfullcredit,creditagreementsummary,True,True,True
5,consumerfullcredit,deliquencyinformation,True,True,True
6,consumerfullcredit,employmenthistory,True,True,True
7,consumerfullcredit,enquirydetails,True,True,True
8,consumerfullcredit,enquiryhistorytop,True,True,True
9,consumerfullcredit,guarantorcount,True,True,True


#### 6.1. At the base node level, we can observe the following nodes or features to play a important factor

1. `accountmonthlypaymenthistory`, -> The payment history of last 24 months and other details about the tyoe of loan it has, opening balance, status. This gives us idea in temporal manner how the observation id is paying the amount and what is the amount overdue on the observation id. This information will play in important role in considering the patterns in payment, value of payment and any chance of future missing the payment.

2. `accountrating` -> Considering this most important as it tells more about what other good loan, bad loan, auto loan good or bad, study loan or bad. etc This is important as it assess the client exposure to other loan and tells about he number of such good loan or bad loan for each segment.

3. `creditaccountsummary` -> The rating variable, account arrear, total accounts, totalmonthlyinstalment and others are important sub nodes withing this features that important to rate and assess the obervation id for risk model. 

4. `creditagreementsummary` -> It has major information about differen credit agreement done by the observation id with other banks and what is the balance at the time of opening the bank account. This tells us how much exposed the observation id is to other credit agreement. This also gives a pciture of observations commitment to pay loan, exposure to loan in pther creditors.

5. `deliquencyinformation` -> It gives idea about the any minor crime or penalty, It give us idea that client may be a risky or non-risky. Generally, this information is sensitive should be handled carefully, but however it is important to note information. 

6. `employmenthistory` -> A important factor, holding information that whether the client is educated or not, profession, employer details. But it also could make our model bias towards the people holding better profession to those not. Which could not be always the case that a better profession person does not deliquent on his/her laoan. We have to look deeper into the analysis of this.

> Most importantly, We have to do EDA on all above variable, perform data validaton, missing value identificatiom, outliers identificaton.


#### 6.2. Below an example how to extract the columns from the base table

In [16]:
# 1. Here we are going to select only that base node which is important in our analysis and pass that as input to our transformed datafrema

important_features = df_features[(df_features['node_1']=='accountrating')]['features'].to_list()
df_imp = df_transformed[important_features]
df_imp.head(3)

,consumerfullcredit.accountrating.noofotheraccountsbad,consumerfullcredit.accountrating.noofotheraccountsgood,consumerfullcredit.accountrating.noofretailaccountsbad,consumerfullcredit.accountrating.noofretailaccountsgood,consumerfullcredit.accountrating.nooftelecomaccountsbad,consumerfullcredit.accountrating.noofautoloanaccountsbad,consumerfullcredit.accountrating.noofautoloanccountsgood,consumerfullcredit.accountrating.noofhomeloanaccountsbad,consumerfullcredit.accountrating.nooftelecomaccountsgood,consumerfullcredit.accountrating.noofhomeloanaccountsgood,consumerfullcredit.accountrating.noofjointloanaccountsbad,consumerfullcredit.accountrating.noofstudyloanaccountsbad,consumerfullcredit.accountrating.noofcreditcardaccountsbad,consumerfullcredit.accountrating.noofjointloanaccountsgood,consumerfullcredit.accountrating.noofstudyloanaccountsgood,consumerfullcredit.accountrating.noofcreditcardaccountsgood,consumerfullcredit.accountrating.noofpersonalloanaccountsbad,consumerfullcredit.accountrating.noofpersonalloanaccountsgood
0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,0,3,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### FINAL FUNCTION . BRINGING ALL TOGETHER

In [17]:
import pandas as pd
import itertools

def convert_json(data, prefix=''):
    """
    This functions take a object as input and checks if the object is dictionary or list, 
    and then recursively flattens it by adding prefix

    Arguments: 
        data : This is the dictionary, or list from the column of dataframe
        prefix = Default -> ''. To se the base prefix or the root node of the dictionary and recursively update it

    Returns:
        rows : A list with key and values for each of the nested key in dictionary or list
    """

    # if the object is dictionary, Change the prefix to prefix + key + '.
    if isinstance(data, dict):
        rows = []
        for key, value in data.items():
            rows += convert_json(value, prefix + key + '.')
        return rows
    
    # if the object is dictionary, Change the prefix to prefix + key + '.
    elif isinstance(data, list):
        rows = []
        for i, item in enumerate(data):
            rows += convert_json(item, prefix + str(i) + '.')
        return rows
    
    # if it str, integer or non-nesteds
    else:
        return [(prefix[:-1], data)]
       

def transform_json_extract_features(df, feature_relevant):
        """
        This function take as dataframe as input and feature you select and returns dataframe with that relevant feature.

        Arguments:
            df : The dataframe of a read json file
            feature_relevant : a string input for the relevant feature

        Returns:        
            df_imp_features : The dataframe with relevant and important features
        """
        rows=[]
        # 1. We will first check if the column requires this transformation or not. Then call the convert json function
        for col in df.columns:
            if df[col].dtype == 'object':
                for item in df[col]:
                    rows.append(dict(convert_json(item)))

            else:
                df_non_flatten = df[col]

        # 2. Storing appended rows output as a dataframe
        df_flatten = pd.DataFrame(rows)

        # 3. Concatenating the two ouput from above loop
        df_transformed = pd.concat([df_non_flatten, df_flatten], axis=1)

        df_features = df_transformed.columns.to_frame(index=None, name='features')

        # 4. Split the column names separated by '.' and store in separate dataframe named df_features
        column_names =[]
        for i in df_features.features.str.split('.', expand=True).columns:
            column_names.append('node_'+ str(i))

        df_features[column_names] = df_features.features.str.split('.', expand=True)

        # 5. Here we are going to select only that base node which is important in our analysis and pass that as input to our transformed datafrema
        important_features = []
        for f in feature_relevant:
            features = df_features[(df_features['node_1']==f)]['features'].to_list()
            important_features.append(features)
        
        flat_features_ls = list(itertools.chain(*important_features))
        df_imp_features = df_transformed[flat_features_ls]
        
        return df_imp_features

### Using the complete funtion and passing inputs

In [18]:
# 1. Loading the data to a dataframe
df = pd.read_json('../dat/credit_report_sample.json')

# 2. Enter features as per your choice and business sense
feature_relevant = ['employmenthistory', 'accountrating', 'creditaccountsummary']

# 3. Run the function.
df_imp_features = transform_json_extract_features(df, feature_relevant)
df_imp_features.head(3)

,consumerfullcredit.employmenthistory.0.occupation,consumerfullcredit.employmenthistory.0.updatedate,consumerfullcredit.employmenthistory.0.updateondate,consumerfullcredit.employmenthistory.0.employerdetail,consumerfullcredit.employmenthistory.1.occupation,consumerfullcredit.employmenthistory.2.occupation,consumerfullcredit.employmenthistory.3.occupation,consumerfullcredit.employmenthistory.3.employerdetail,consumerfullcredit.employmenthistory.4.occupation,consumerfullcredit.employmenthistory.1.employerdetail,...,consumerfullcredit.creditaccountsummary.totaldishonouredamount,consumerfullcredit.creditaccountsummary.totalmonthlyinstalment,consumerfullcredit.creditaccountsummary.totalnumberofjudgement,consumerfullcredit.creditaccountsummary.totaldishonouredamount1,consumerfullcredit.creditaccountsummary.totalmonthlyinstalment1,consumerfullcredit.creditaccountsummary.totalnumberofjudgement1,consumerfullcredit.creditaccountsummary.totalnumberofdishonoured,consumerfullcredit.creditaccountsummary.totalnumberofdishonoured1,consumerfullcredit.creditaccountsummary.totalaccountingodcondition,consumerfullcredit.creditaccountsummary.totalaccountingodcondition1
0,PUBLIC SERVANTS,16/09/2016,16/09/2016,ALL MILITARY STAFFS,PUBLIC SERVANTS,PUBLIC SERVANTS,Army Police Defence,Nigerian navy,PUBLIC SERVANTS,NaN,...,0.00,"77,404.00",0,0.00,0.00,0,0,0,0,0
1,CIVIL SERVANT,21/11/2014,21/11/2014,None,DOCTOR,DOCTOR,DOCTOR,MINISTRY OF WOMEN AFFAIRS AND SOCIAL DEVELOPMENT,DOCTOR,MINISTRY OF WOMEN AFFAIRS AND SOCIAL DEVELOPME...,...,0.00,"132,176.00",0,0.00,0.00,0,0,0,0,0
2,STUDENT,NaN,NaN,NaN,STUDENT,STUDENT,STUDENT,NaN,STUDENT,NaN,...,0.00,"7,000.00",0,0.00,0.00,0,0,0,0,0


### PLEASE PROVIDE YOUR VALUABLE FEEDBACK